# 제6장: 일본어 DNN 음성 합성 시스템 구현

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/r9y9/ttslearn/blob/master/notebooks/ch06_Recipe-DNNTTS.ipynb)

Google colab에서 실행하는 데 예상되는 소요 시간: 1시간

이 노트북의 레시피 설정은 Google Colab에서 실행할 때 시간 초과를 피하기 위해 학습 조건을 책에 나열된 설정에서 일부 수정한 것입니다. (배치 크기 줄이기 등).
참고로 책에 기재된 조건으로 저자(야마모토)가 레시피를 실행한 결과를 아래에서 공개하고 있습니다.

- Tensorboard logs: https://tensorboard.dev/experiment/ajmqiymoTx6rADKLF8d6sA/
- exp 디렉토리 (학습 모델, 합성 음성 포함) : https://drive.google.com/file/d/171gGoH3H4PJ-9cMQES-l6KpTu9n0udGD/view?usp=sharing (12.8 MB)

## 준비

### Google Colab을 이용하는 경우

Google Colab에서 이 노트북을 실행하려면 메뉴의 '런타임 -> 런타임 시간 변경'에서 '하드웨어 가속기'를 **GPU**로 변경하세요.

### Python version

In [ ]:
!python -VV

### ttslearn 설치

In [ ]:
%%capture
try:
    import ttslearn
except ImportError:
    !pip install ttslearn

In [ ]:
import ttslearn
ttslearn.__version__

## 6.1 이 장의 일본어 음성 합성 시스템 구현

### 학습된 모델을 이용한 음성 합성

In [ ]:
from ttslearn.dnntts import DNNTTS
from IPython.display import Audio

engine = DNNTTS()
wav, sr = engine.tts("深層学習に基づく音声合成システムです。")
Audio(wav, rate=sr)

In [ ]:
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8,2))
librosa.display.waveshow(wav.astype(np.float32), sr, ax=ax)
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Amplitude")
plt.tight_layout()

### 레시피 실행 전 준비

In [ ]:
%%capture
from ttslearn.env import is_colab
from os.path import exists

# pip install ttslearn은 레시피를 설치하지 않으므로 수동으로 다운로드
if is_colab() and not exists("recipes.zip"):
    !curl -LO https://github.com/r9y9/ttslearn/releases/download/v{ttslearn.__version__}/recipes.zip
    !unzip -o recipes.zip

In [ ]:
import os
# recipe 디렉토리로 이동
cwd = os.getcwd()
if cwd.endswith("notebooks"):
    os.chdir("../recipes/dnntts/")
elif is_colab():
    os.chdir("recipes/dnntts/")

In [ ]:
import time
start_time = time.time()

### 패키지 임포트

In [ ]:
%pylab inline
%load_ext autoreload
%load_ext tensorboard
%autoreload
import IPython
from IPython.display import Audio
import tensorboard as tb
import os

In [ ]:
# 수치 연산
import numpy as np
import torch
from torch import nn
# 음성 파형 불러오기
from scipy.io import wavfile
#  풀 컨텍스트 라벨, 질의 파일 로드
from nnmnkwii.io import hts
# 음성 분석
import pysptk
import pyworld
# 음성 분석, 시각화
import librosa
import librosa.display
import pandas as pd
# 파이썬에서 배우는 음성 합성
import ttslearn

In [ ]:
# 시드 고정
from ttslearn.util import init_seed
init_seed(773)

In [ ]:
torch.__version__

### 그래프 그리기 설정 (描画周りの設定) // 번역 수정 필요

In [ ]:
from ttslearn.notebook import get_cmap, init_plot_style, savefig
cmap = get_cmap()
init_plot_style()

### 레시피 설정

In [ ]:
# run.sh를 사용하여 학습 스크립트를 노트북에서 실행하려면 True
# google colab의 경우 True라고 가정합니다.
# 로컬 환경의 경우 run.sh를 터미널에서 실행하는 것이 좋습니다.
# 이 경우이 노트북은 시각화 및 학습 된 모델을 테스트하는 데 사용됩니다.
run_sh = is_colab()

# CUDA
# NOTE: run.sh의 인수로 전달하기 때문에 bool이 아닌 문자열로 정의합니다.
cudnn_benchmark = "true"
cudnn_deterministic = "false"

# 특징(feature) 추출시 병렬 처리 작업 수
n_jobs = os.cpu_count()//2

# 연속 길이 모델의 설정 파일 이름
duration_config_name="duration_dnn"
# 음향 모델 설정 파일 이름
acoustic_config_name="acoustic_dnn_sr16k"

# 연속 길이 모델 및 음향 모델 학습의 배치 크기
batch_size = 32
# 지속 길이 모델 및 음향 모델 학습의 에포크 수
# 참고: 계산 시간을 줄이기 위해 다소 적게 설정했습니다. 품질을 높이려면 30-50 에포크 수를 사용해보십시오.
nepochs = 10

# run.sh를 통해 실행되는 스크립트의 tqdm
run_sh_tqdm = "none"

In [ ]:
# 노트북에서 사용하는 테스트용 발화(학습 데이터, 평가 데이터)
train_utt = "BASIC5000_0001"
test_utt = "BASIC5000_5000"

### Tensorboard로 로그 시각화

In [ ]:
# 노트북에서 tensorboard 로깅을 확인하려면 다음 행을 활성화하십시오.
if is_colab():
    %tensorboard --logdir tensorboard/

## 6.2 프로그램 구현 전 준비

### stage -1: 코퍼스 다운로드

In [ ]:
if is_colab():
    ! ./run.sh --stage -1 --stop-stage -1

### Stage 0: 학습/검증/평가 데이터 분할

In [ ]:
if run_sh:
    ! ./run.sh --stage 0 --stop-stage 0

In [ ]:
! ls data/

In [ ]:
! head data/dev.list

## 6.3 연속 길이 모델에 대한 전처리

### 연속 길이 모델을 위한 1 발화에 대한 전처리

In [ ]:
import ttslearn
from nnmnkwii.io import hts
from nnmnkwii.frontend import merlin as fe

# 언어 특징량 추출에 사용하기 위한 질문 파일
binary_dict, numeric_dict = hts.load_question_set(ttslearn.util.example_qst_file())

# 음성의 풀 컨텍스트 라벨 로드
labels = hts.load(ttslearn.util.example_label_file())

# 연속 길이 모델 입력: 언어 특징량
in_feats = fe.linguistic_features(labels, binary_dict, numeric_dict)

# 연속 길이 모델 출력: 음소 연속 길이
out_feats = fe.duration_features(labels)

In [ ]:
print("入力特徴量のサイズ:", in_feats.shape)
print("出力特徴量のサイズ:", out_feats.shape)

In [ ]:
# 시각화를 위해 정규화
in_feats_norm = in_feats / np.maximum(1, np.abs(in_feats).max(0))
fig, ax = plt.subplots(2, 1, figsize=(8,6))
ax[0].set_title("Duration model's input: linguistic features")
ax[1].set_title("Duration model's output: phoneme durations")
ax[0].imshow(in_feats_norm.T, aspect="auto", interpolation="nearest", origin="lower", cmap=cmap)
ax[0].set_ylabel("Context")

ax[1].bar(np.arange(len(out_feats)), out_feats.reshape(-1))

for a in ax:
    a.set_xlim(-0.5, len(in_feats)-0.5)
    a.set_xlabel("Phoneme")
ax[1].set_ylabel("Duration (the number of frames)")
plt.tight_layout()

# 그림 6-3
savefig("fig/dnntts_impl_duration_inout")

### 레시피의 stage 1 실행

배치 처리를 실시하는 커멘드 라인 프로그램은, `preprocess_duration.py`를 참조해 주세요.

In [ ]:
if run_sh:
    ! ./run.sh --stage 1 --stop-stage 1 --n-jobs $n_jobs 

## 6.4: 음향 모델을 위한 전처리

### 음향 모델을 위한 1 발화에 대한 전처리

In [ ]:
from ttslearn.dsp import world_spss_params

# 언어 특징량 추출에 사용하기 위한 질의 파일
binary_dict, numeric_dict = hts.load_question_set(ttslearn.util.example_qst_file())

# 음성의 풀 컨텍스트 라벨 로드
labels = hts.load(ttslearn.util.example_label_file())

# 음향 모델 입력: 언어 특징량
in_feats = fe.linguistic_features(labels, binary_dict, numeric_dict, add_frame_features=True, subphone_features="coarse_coding")

# 음성 파일 읽기
_sr, x = wavfile.read(ttslearn.util.example_audio_file())
sr = 16000
x = (x / 32768).astype(np.float64)
x = librosa.resample(x, _sr, sr)

# 음향 모델 출력: 음향 특징량
out_feats = world_spss_params(x, sr)

# 프레임 수 조정
minL = min(in_feats.shape[0], out_feats.shape[0])
in_feats, out_feats = in_feats[:minL], out_feats[:minL]

# 시작과 끝의 음성이 아닌 구간의 길이 조정
assert "sil" in labels.contexts[0] and "sil" in labels.contexts[-1]
start_frame = int(labels.start_times[1] / 50000)
end_frame = int(labels.end_times[-2] / 50000)

# 시작: 50ms, 후미: 100ms
start_frame = max(0, start_frame - int(0.050 / 0.005))
end_frame = min(minL, end_frame + int(0.100 / 0.005))

in_feats = in_feats[start_frame:end_frame]
out_feats = out_feats[start_frame:end_frame]

In [ ]:
print("입력 특징량의 크기:", in_feats.shape)
print("출력 특징량의 크기:", out_feats.shape)

#### 음향 특징량을 분리하여 확인

In [ ]:
from ttslearn.dnntts.multistream import get_static_features

sr = 16000
hop_length = int(sr * 0.005)
alpha = pysptk.util.mcepalpha(sr)
fft_size = pyworld.get_cheaptrick_fft_size(sr)

# 동적 특징량을 제외하고 각 음향 특징량을 꺼내
mgc, lf0, vuv, bap = get_static_features(
    out_feats, num_windows=3, stream_sizes=[120, 3, 1, 3],
    has_dynamic_features=[True, True, False, True]) 
print("멜 캡스트럼의 크기:", mgc.shape)
print("연속 로그 기본 주파수의 크기:", lf0.shape)
print("유성/무성 플래그의 크기:", vuv.shape)
print("대역 비주기성 지표의 크기:", bap.shape)

In [ ]:
def vis_out_feats(mgc, lf0, vuv, bap):
    fig, ax = plt.subplots(3, 1, figsize=(8,8))
    ax[0].set_title("Spectral envelope")
    ax[1].set_title("Fundamental frequency")
    ax[2].set_title("Aperiodicity")
    
    logsp = np.log(pysptk.mc2sp(mgc, alpha, fft_size))
    librosa.display.specshow(logsp.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="hz", cmap=cmap, ax=ax[0])
    
    timeaxis = np.arange(len(lf0)) * 0.005
    f0 = np.exp(lf0)
    f0[vuv < 0.5] = 0
    ax[1].plot(timeaxis, f0, linewidth=2)
    ax[1].set_xlim(0, len(f0)*0.005)

    aperiodicity = pyworld.decode_aperiodicity(bap.astype(np.float64), sr, fft_size)
    librosa.display.specshow(aperiodicity.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="hz", cmap=cmap, ax=ax[2])
    
    for a in ax:
        a.set_xlabel("Time [sec]")
        a.set_ylabel("Frequency [Hz]")
        # 말미의 비음성 구간 제외
        a.set_xlim(0, 2.55)
    
    plt.tight_layout()

# 음향 특징량의 시각화
vis_out_feats(mgc, lf0, vuv, bap)
# 그림 6-4
savefig("./fig/dnntts_impl_acoustic_out_feats")

#### 음향 모델 입력 및 출력 시각화

In [ ]:
# 시각화를 위해 정규화
from scipy.stats import zscore 

in_feats_norm = in_feats / np.maximum(1, np.abs(in_feats).max(0))
out_feats_norm = zscore(out_feats)

fig, ax = plt.subplots(2, 1, figsize=(8,6))
ax[0].set_title("Acoustic model's input: linguistic features")
ax[1].set_title("Acoustic model's output: acoustic features")
mesh = librosa.display.specshow(
    in_feats_norm.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="frames", ax=ax[0], cmap=cmap)
fig.colorbar(mesh, ax=ax[0])
mesh = librosa.display.specshow(
    out_feats_norm.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="frames",ax=ax[1], cmap=cmap)
# NOTE: 실제로는 [-4, 4] 범위를 벗어난 값이 있지만 시인성을 위해 [-4, 4]로 설정합니다.
mesh.set_clim(-4, 4)
fig.colorbar(mesh, ax=ax[1])

ax[0].set_ylabel("Context")
ax[1].set_ylabel("Feature")

for a in ax:
    a.set_xlabel("Time [sec]")
    # 말미의 비음성 구간 제외
    a.set_xlim(0, 2.55)
    
plt.tight_layout()

### 레시피의 stage 2 실행

배치 처리를 실시하는 커멘드 라인 프로그램은, `preprocess_acoustic.py`를 참조해 주세요.

In [ ]:
if run_sh:
    ! ./run.sh --stage 2 --stop-stage 2 --n-jobs $n_jobs

## 6.5 특징량 정규화

정규화를 위한 통계량을 계산하는 명령행 프로그램은 `recipes/common/fit_scaler.py`를 참조하십시오. 또, 정규화를 실시하는 커멘드 라인 프로그램은, `recipes/common/preprocess_normalize.py` 를 참조해 주세요.

### 레시피의 stage 3 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 3 --stop-stage 3 --n-jobs $n_jobs

### 정규화 처리 결과 확인

In [ ]:
# 언어 특징량 정규화 전후
in_feats = np.load(f"dump/jsut_sr16000/org/train/in_acoustic/{train_utt}-feats.npy")
in_feats_norm = np.load(f"dump/jsut_sr16000/norm/train/in_acoustic/{train_utt}-feats.npy")
fig, ax = plt.subplots(2, 1, figsize=(8,6))
ax[0].set_title("Linguistic features (before normalization)")
ax[1].set_title("Linguistic features (after normalization)")
mesh = librosa.display.specshow(
    in_feats.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="frames", ax=ax[0], cmap=cmap)
fig.colorbar(mesh, ax=ax[0])
mesh = librosa.display.specshow(
    in_feats_norm.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="frames",ax=ax[1], cmap=cmap)
# 참고 : 실제로는 [-4, 4] 범위를 벗어난 값이 있지만 가시성을 위해 [-4, 4]로 설정합니다.
mesh.set_clim(-4, 4)
fig.colorbar(mesh, ax=ax[1])

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Context")
    # 말미의 비음성 구간 제외
    a.set_xlim(0, 2.55) 
plt.tight_layout()
# 그림 6-5
savefig("./fig/dnntts_impl_in_feats_norm")

In [ ]:
# 음향 특징량의 정규화 전후
out_feats = np.load(f"dump/jsut_sr16000/org/train/out_acoustic/{train_utt}-feats.npy")
out_feats_norm = np.load(f"dump/jsut_sr16000/norm/train/out_acoustic/{train_utt}-feats.npy")
fig, ax = plt.subplots(2, 1, figsize=(8,6))
ax[0].set_title("Acoustic features (before normalization)")
ax[1].set_title("Acoustic features (after normalization)")
mesh = librosa.display.specshow(
    out_feats.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="frames", ax=ax[0], cmap=cmap)
fig.colorbar(mesh, ax=ax[0])
mesh = librosa.display.specshow(
    out_feats_norm.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="frames",ax=ax[1], cmap=cmap)
# NOTE : 실제로는 [-4, 4] 범위를 벗어난 값이 있지만 가시성을 위해 [-4, 4]로 설정합니다.
mesh.set_clim(-4, 4)
fig.colorbar(mesh, ax=ax[1])

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Feature")
    # 말미의 비음성 구간 제외
    a.set_xlim(0, 2.55)
plt.tight_layout()

## 6.6 신경망 구현

### 완전 결합형 신경망

In [ ]:
class DNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_layers=2):
        super(DNN, self).__init__()
        model = [nn.Linear(in_dim, hidden_dim), nn.ReLU()]
        for _ in range(num_layers):
            model.append(nn.Linear(hidden_dim, hidden_dim))
            model.append(nn.ReLU())
        model.append(nn.Linear(hidden_dim, out_dim))
        self.model = nn.Sequential(*model)

    def forward(self, x, lens=None):
        return self.model(x)

In [ ]:
DNN(in_dim=325, hidden_dim=64, out_dim=1, num_layers=2)

### LSTM-RNN

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class LSTMRNN(nn.Module):
    def __init__(
        self, in_dim, hidden_dim, out_dim, num_layers=1, bidirectional=True, dropout=0.0
    ):
        super(LSTMRNN, self).__init__()
        self.num_layers = num_layers
        num_direction = 2 if bidirectional else 1
        self.lstm = nn.LSTM(
            in_dim,
            hidden_dim,
            num_layers,
            bidirectional=bidirectional,
            batch_first=True,
            dropout=dropout,
        )
        self.hidden2out = nn.Linear(num_direction * hidden_dim, out_dim)

    def forward(self, seqs, lens):
        seqs = pack_padded_sequence(seqs, lens, batch_first=True)
        out, _ = self.lstm(seqs)
        out, _ = pad_packed_sequence(out, batch_first=True)
        out = self.hidden2out(out)
        return out

In [ ]:
LSTMRNN(in_dim=325, hidden_dim=64, out_dim=1, num_layers=2)

## 6.7 학습 스크립트 구현

### DataLoader 구현

#### Dataset 클래스 정의

In [ ]:
from torch.utils import data as data_utils

class Dataset(data_utils.Dataset):
    def __init__(self, in_paths, out_paths):
        self.in_paths = in_paths
        self.out_paths = out_paths

    def __getitem__(self, idx):
        return np.load(self.in_paths[idx]), np.load(self.out_paths[idx])

    def __len__(self):
        return len(self.in_paths)

#### DataLoader 사용 예

In [ ]:
from pathlib import Path
from ttslearn.util import pad_2d

def collate_fn_dnntts(batch):
    lengths = [len(x[0]) for x in batch]
    max_len = max(lengths)
    x_batch = torch.stack([torch.from_numpy(pad_2d(x[0], max_len)) for x in batch])
    y_batch = torch.stack([torch.from_numpy(pad_2d(x[1], max_len)) for x in batch])
    lengths = torch.tensor(lengths, dtype=torch.long)
    return x_batch, y_batch, lengths


in_paths = sorted(Path("./dump/jsut_sr16000/norm/dev/in_duration/").glob("*.npy"))
out_paths = sorted(Path("./dump/jsut_sr16000/norm/dev/out_duration/").glob("*.npy"))

dataset = Dataset(in_paths, out_paths)
data_loader = data_utils.DataLoader(dataset, batch_size=8, collate_fn=collate_fn_dnntts, num_workers=0)

in_feats, out_feats, lengths = next(iter(data_loader))

print("입력 특징량의 크기:", tuple(in_feats.shape))
print("출력 특징량의 크기:", tuple(out_feats.shape))
print("계열 길이의 크기:", tuple(lengths.shape))
# 계열: seq2seq에서 sequence에 해당하는 단어로 보입니다.

#### ミニバッチの可視化

In [ ]:
fig, ax = plt.subplots(len(in_feats), 1, figsize=(8,10), sharex=True, sharey=True)
for n in range(len(in_feats)):
    x = in_feats[n].data.numpy()
    mesh = ax[n].imshow(x.T, aspect="auto", interpolation="nearest", origin="lower", cmap=cmap)
    fig.colorbar(mesh, ax=ax[n])
    # NOTE : 실제로는 [-4, 4] 범위를 벗어난 값이 있지만 가시성을 위해 [-4, 4]로 설정합니다.
    mesh.set_clim(-4, 4)
    
ax[-1].set_xlabel("Phoneme")
for a in ax:
    a.set_ylabel("Context")
plt.tight_layout()

# 그림 6-6
savefig("fig/dnntts_impl_minibatch")

### 학습 전 준비

In [ ]:
from ttslearn.dnntts import DNN
from torch import optim

model = DNN(in_dim=325, hidden_dim=64, out_dim=1, num_layers=2)

# lr은 학습률을 나타냅니다.
optimizer = optim.Adam(model.parameters(), lr=0.01)

# gamma는 학습률의 감쇠 계수를 나타냅니다.
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, gamma=0.5, step_size=10)

### 학습 루프 구현

In [ ]:
#DataLoader를 사용하여 미니 배치 생성 : 미니 일괄 처리
for in_feats, out_feats, lengths in data_loader:
    # 순전파 계산
    pred_out_feats = model(in_feats, lengths)
    # 손실 계산
    loss = nn.MSELoss()(pred_out_feats, out_feats)
    # 손실 값을 출력
    print(loss.item())
    # optimizer에 축적 된 기울기를 재설정
    optimizer.zero_grad()
    # 오차의 역전파 계산
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()

### hydra를 이용한 커맨드 라인 프로그램 구현

`hydra/hydra_quick_start`와 `hydra/hydra_composision`을 참조하십시오.

### hydra를 사용한 실용적인 학습 스크립트 구성 파일

`conf/train_dnntts` 디렉토리를 참조하십시오.

In [ ]:
! cat conf/train_dnntts/config.yaml

### hydra를 사용하여 실용적인 학습 스크립트 구현

`train_dnntts.py`를 참조하십시오.

## 6.8 연속 길이 모델 학습

### 연속 길이 모델의 설정 파일

In [ ]:
! cat conf/train_dnntts/model/{duration_config_name}.yaml

### 연속 길이 모델 인스턴스화

In [ ]:
import hydra
from omegaconf import OmegaConf
hydra.utils.instantiate(OmegaConf.load(f"conf/train_dnntts/model/{duration_config_name}.yaml").netG)

### 레시피 stage 4 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 4 --stop-stage 4 --duration-model $duration_config_name \
    --tqdm $run_sh_tqdm --dnntts-data-batch-size $batch_size --dnntts-train-nepochs $nepochs \
    --cudnn-benchmark $cudnn_benchmark --cudnn-deterministic $cudnn_deterministic

### 손실 함수의 값 추이

저자에 의한 실험 결과입니다. Tensorboard 로그는 https://tensorboard.dev/에 업로드되었습니다.
로그 데이터를 `tensorboard` 패키지를 이용해 다운로드합니다.

https://tensorboard.dev/experiment/ajmqiymoTx6rADKLF8d6sA/

In [ ]:
if exists("tensorboard/all_log.csv"):
    df = pd.read_csv("tensorboard/all_log.csv")
else:
    experiment_id = "ajmqiymoTx6rADKLF8d6sA"
    experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
    df = experiment.get_scalars(pivot=True) 
    df.to_csv("tensorboard/all_log.csv", index=False)
df["run"].unique()

In [ ]:
duration_loss = df[df.run.str.contains("duration")]

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(duration_loss["step"], duration_loss["Loss/train"], label="Train")
ax.plot(duration_loss["step"], duration_loss["Loss/dev"], "--", label="Dev")
ax.set_xlabel("Epoch")
ax.set_ylabel("Epoch loss")
plt.legend()

# 그림 6-8
savefig("fig/dnntts_impl_duration_dnn_loss")

## 6.9 음향 모델 학습

### 음향 모델 설정 파일

In [ ]:
! cat conf/train_dnntts/model/{acoustic_config_name}.yaml

### 음향 모델 인스턴스화

In [ ]:
import hydra
from omegaconf import OmegaConf
hydra.utils.instantiate(OmegaConf.load(f"conf/train_dnntts/model/{acoustic_config_name}.yaml").netG)

### 레시피의 stage 5 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 5 --stop-stage 5 --acoustic-model $acoustic_config_name \
    --tqdm $run_sh_tqdm --dnntts-data-batch-size $batch_size --dnntts-train-nepochs $nepochs \
    --cudnn-benchmark $cudnn_benchmark --cudnn-deterministic $cudnn_deterministic

### 손실 함수의 값 추이

In [ ]:
acoustic_loss = df[df.run.str.contains("acoustic")]

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(acoustic_loss["step"], acoustic_loss["Loss/train"], label="Train")
ax.plot(acoustic_loss["step"], acoustic_loss["Loss/dev"], "--", label="Dev")
ax.set_xlabel("Epoch")
ax.set_ylabel("Epoch loss")

plt.legend()

# 그림 6-9
savefig("fig/dnntts_impl_acoustic_dnn_loss")

## 6.10 학습된 모델을 사용하여 텍스트에서 음성 합성

### 학습된 모델 로드

In [ ]:
import joblib
device = torch.device("cpu")

#### 연속 길이 모델 로드

In [ ]:
duration_config = OmegaConf.load(f"exp/jsut_sr16000/{duration_config_name}/model.yaml")
duration_model = hydra.utils.instantiate(duration_config.netG)
checkpoint = torch.load(f"exp/jsut_sr16000/{duration_config_name}/latest.pth", map_location=device)
duration_model.load_state_dict(checkpoint["state_dict"])
duration_model.eval();

#### 음향 모델 로드

In [ ]:
acoustic_config = OmegaConf.load(f"exp/jsut_sr16000/{acoustic_config_name}/model.yaml")
acoustic_model = hydra.utils.instantiate(acoustic_config.netG)
checkpoint = torch.load(f"exp/jsut_sr16000/{acoustic_config_name}/latest.pth", map_location=device)
acoustic_model.load_state_dict(checkpoint["state_dict"])
acoustic_model.eval();

#### 통계량 로드

In [ ]:
duration_in_scaler = joblib.load("./dump/jsut_sr16000/norm/in_duration_scaler.joblib")
duration_out_scaler = joblib.load("./dump/jsut_sr16000/norm/out_duration_scaler.joblib")
acoustic_in_scaler = joblib.load("./dump/jsut_sr16000/norm/in_acoustic_scaler.joblib")
acoustic_out_scaler = joblib.load("./dump/jsut_sr16000/norm/out_acoustic_scaler.joblib")

### 음소 연속 길이 예측

In [ ]:
@torch.no_grad()
def predict_duration(
    device,  # cpu or cuda
    labels,  # 풀 컨텍스트 라벨
    duration_model,  # 학습된 연속 길이 모델
    duration_config,  # 연속 길이 모델 설정
    duration_in_scaler,  # 언어 특징량 정규화용 StandardScaler
    duration_out_scaler,  # 음소 연속 길이 정규화용 StandardScaler
    binary_dict,  # 이진 특징량을 추출하는 정규식
    numeric_dict,  # 수치 특징량을 추출하는 정규 표현
):
    # 언어 특징량 추출
    in_feats = fe.linguistic_features(labels, binary_dict, numeric_dict).astype(np.float32)

    # 언어 특징량 정규화
    in_feats = duration_in_scaler.transform(in_feats)

    # 지속 길이 예측
    x = torch.from_numpy(in_feats).float().to(device).view(1, -1, in_feats.shape[-1])
    pred_durations = duration_model(x, [x.shape[1]]).squeeze(0).cpu().data.numpy()

    # 예측된 연속 길이에 대해 정규화를 역변환합니다.
    pred_durations = duration_out_scaler.inverse_transform(pred_durations)

    # 임계값 처리
    pred_durations[pred_durations <= 0] = 1
    pred_durations = np.round(pred_durations)

    return pred_durations

In [ ]:
from ttslearn.util import lab2phonemes, find_lab, find_feats

labels = hts.load(find_lab("downloads/jsut_ver1.1/", test_utt))

# 풀 컨텍스트 라벨에서 음소만 추출
test_phonemes = lab2phonemes(labels)

# 언어 특징량 추출에 사용하기 위한 질의 파일
binary_dict, numeric_dict = hts.load_question_set(ttslearn.util.example_qst_file())

# 음소 연속 길이 예측
durations_test = predict_duration(
    device, labels, duration_model, duration_config, duration_in_scaler, duration_out_scaler,
    binary_dict, numeric_dict)
durations_test_target = np.load(find_feats("dump/jsut_sr16000/org", test_utt, typ="out_duration"))

fig, ax = plt.subplots(1,1, figsize=(6,4))
ax.plot(durations_test_target, "-+", label="Target")
ax.plot(durations_test, "--*", label="Predicted")
ax.set_xticks(np.arange(len(test_phonemes)))
ax.set_xticklabels(test_phonemes)
ax.set_xlabel("Phoneme")
ax.set_ylabel("Duration (the number of frames)")
ax.legend()

plt.tight_layout()

# 그림 6-10
savefig("fig/dnntts_impl_duration_comp")

### 음향 특징량 예측

In [ ]:
from ttslearn.dnntts.multistream import get_windows, multi_stream_mlpg

@torch.no_grad()
def predict_acoustic(
    device,  # CPU or GPU
    labels,  # 풀 컨텍스트 라벨
    acoustic_model,  # 학습된 음향 모델
    acoustic_config,  # 음향 모델 설정
    acoustic_in_scaler,  # 언어 특징량 정규화용 StandardScaler
    acoustic_out_scaler,  # 음향 특징량의 정규화용 StandardScaler
    binary_dict,  # 이진 특징량을 추출하는 정규식
    numeric_dict,  # 수치 특징량을 추출하는 정규 표현
    mlpg=True,  # MLPG 사용 여부
):
    # 프레임별 언어 특징량 추출
    in_feats = fe.linguistic_features(
        labels,
        binary_dict,
        numeric_dict,
        add_frame_features=True,
        subphone_features="coarse_coding",
    )
    # 정규화
    in_feats = acoustic_in_scaler.transform(in_feats)

    # 음향 특징량 예측
    x = torch.from_numpy(in_feats).float().to(device).view(1, -1, in_feats.shape[-1])
    pred_acoustic = acoustic_model(x, [x.shape[1]]).squeeze(0).cpu().data.numpy()

    # 예측된 음향 특징량에 대해 정규화의 역변환을 실시합니다.
    pred_acoustic = acoustic_out_scaler.inverse_transform(pred_acoustic)

    # 파라미터 생성 알고리즘(MLPG) 실행
    if mlpg and np.any(acoustic_config.has_dynamic_features):
        # (T, D_out) -> (T, static_dim)
        pred_acoustic = multi_stream_mlpg(
            pred_acoustic,
            acoustic_out_scaler.var_,
            get_windows(acoustic_config.num_windows),
            acoustic_config.stream_sizes,
            acoustic_config.has_dynamic_features,
        )

    return pred_acoustic

In [ ]:
labels = hts.load(f"./downloads/jsut_ver1.1/basic5000/lab/{test_utt}.lab")

# 음향 특징량 예측
out_feats = predict_acoustic(
    device, labels, acoustic_model, acoustic_config, acoustic_in_scaler,
    acoustic_out_scaler, binary_dict, numeric_dict)

In [ ]:
from ttslearn.util import trim_silence
from ttslearn.dnntts.multistream import split_streams

# 특징량은, 전처리로 서두와 말미에 비음성 구간이 잘려져 있기 때문에, 비교를 위해 여기에서도 같은 처리를 실시합니다
out_feats = trim_silence(out_feats, labels)
# 결합된 특징량 분리
mgc_gen, lf0_gen, vuv_gen, bap_gen = split_streams(out_feats, [40, 1, 1, 1])

In [ ]:
# 비교를 위해 자연음성에서 추출한 음향 특징량을 읽기
feats = np.load(f"./dump/jsut_sr16000/org/eval/out_acoustic/{test_utt}-feats.npy")
# 특징량 분리
mgc_ref, lf0_ref, vuv_ref, bap_ref = get_static_features(
    feats, acoustic_config.num_windows, acoustic_config.stream_sizes, acoustic_config.has_dynamic_features)

#### 스펙트럼 포락(envelope)의 시각화

In [ ]:
# 음향 특징을 WORLD의 음성 파라미터로 변환

# 멜 켑스트럼(MFCC)에서 스펙트럼 포락(envelope)으로의 변환
sp_gen= pysptk.mc2sp(mgc_gen, alpha, fft_size)
sp_ref= pysptk.mc2sp(mgc_ref, alpha, fft_size)

mindb = min(np.log(sp_ref).min(), np.log(sp_gen).min())
maxdb = max(np.log(sp_ref).max(), np.log(sp_gen).max())

fig, ax = plt.subplots(2, 1, figsize=(8,6))
mesh = librosa.display.specshow(np.log(sp_ref).T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="linear", cmap=cmap, ax=ax[0])
mesh.set_clim(mindb, maxdb)
fig.colorbar(mesh, ax=ax[0], format="%+2.fdB")
mesh = librosa.display.specshow(np.log(sp_gen).T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="linear", cmap=cmap, ax=ax[1])
mesh.set_clim(mindb, maxdb)
fig.colorbar(mesh, ax=ax[1], format="%+2.fdB")
for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Frequency [Hz]")
    
ax[0].set_title("Spectral envelope of natural speech")
ax[1].set_title("Spectral envelope of generated speech")

plt.tight_layout()

# 그림 6-11
savefig("./fig/dnntts_impl_spec_comp")

#### F0 시각화

In [ ]:
# 로그 기본 주파수에서 기본 주파수로 변환
f0_ref = np.exp(lf0_ref)
f0_ref[vuv_ref < 0.5] = 0
f0_gen = np.exp(lf0_gen)
f0_gen[vuv_gen < 0.5] = 0

timeaxis = librosa.frames_to_time(np.arange(len(f0_ref)), sr=sr, hop_length=int(0.005 * sr))

fix, ax = plt.subplots(1,1, figsize=(8,3))
ax.plot(timeaxis, f0_ref, linewidth=2, label="F0 of natural speech")
ax.plot(timeaxis, f0_gen, "--", linewidth=2, label="F0 of generated speech")

ax.set_xlabel("Time [sec]")
ax.set_ylabel("Frequency [Hz]")
ax.set_xlim(timeaxis[0], timeaxis[-1])

plt.legend()
plt.tight_layout()

# 그림 6-12
savefig("./fig/dnntts_impl_f0_comp")

#### 대역 비주기성 지표 시각화 (bonus)

In [ ]:
# 대역 비주기성 지표
ap_ref = pyworld.decode_aperiodicity(bap_ref.astype(np.float64), sr, fft_size)
ap_gen = pyworld.decode_aperiodicity(bap_gen.astype(np.float64), sr, fft_size)

mindb = min(np.log(ap_ref).min(), np.log(ap_gen).min())
maxdb = max(np.log(ap_ref).max(), np.log(ap_gen).max())

fig, ax = plt.subplots(2, 1, figsize=(8,6))
mesh = librosa.display.specshow(np.log(ap_ref).T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="linear", cmap=cmap, ax=ax[0])
mesh.set_clim(mindb, maxdb)
fig.colorbar(mesh, ax=ax[0])
mesh = librosa.display.specshow(np.log(ap_gen).T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="linear", cmap=cmap, ax=ax[1])
mesh.set_clim(mindb, maxdb)
fig.colorbar(mesh, ax=ax[1])

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Frequency [Hz]")
plt.tight_layout()

### 음성 파형 생성

In [ ]:
from nnmnkwii.postfilters import merlin_post_filter
from ttslearn.dnntts.multistream import get_static_stream_sizes

def gen_waveform(
    sample_rate,  # 샘플링 주파수
    acoustic_features,  # 음향 특징량
    stream_sizes,  # 스트림 크기
    has_dynamic_features,  # 음향 특징량이 동적 특징량을 포함하는지 여부
    num_windows=3,  # 동적 특징량을 계산하는 데 사용되는 창 개수
    post_filter=False,  # 포먼트 강조 포스트 필터를 사용할지 여부
):
    # 정적 특징량의 차원수를 취득
    if np.any(has_dynamic_features):
        static_stream_sizes = get_static_stream_sizes(
            stream_sizes, has_dynamic_features, num_windows
        )
    else:
        static_stream_sizes = stream_sizes

    # 결합된 음향 특징량을 스트림별로 분리
    mgc, lf0, vuv, bap = split_streams(acoustic_features, static_stream_sizes)

    fftlen = pyworld.get_cheaptrick_fft_size(sample_rate)
    alpha = pysptk.util.mcepalpha(sample_rate)

    # 포먼트 강조 포스트 필터
    if post_filter:
        mgc = merlin_post_filter(mgc, alpha)

    # 음향 특징량을 음성 파라미터로 변환
    spectrogram = pysptk.mc2sp(mgc, fftlen=fftlen, alpha=alpha)
    aperiodicity = pyworld.decode_aperiodicity(
        bap.astype(np.float64), sample_rate, fftlen
    )
    f0 = lf0.copy()
    f0[vuv < 0.5] = 0
    f0[np.nonzero(f0)] = np.exp(f0[np.nonzero(f0)])

    # WORLD 보코더를 이용한 음성 생성
    gen_wav = pyworld.synthesize(
        f0.flatten().astype(np.float64),
        spectrogram.astype(np.float64),
        aperiodicity.astype(np.float64),
        sample_rate,
    )

    return gen_wav

### 모든 모델을 결합하여 음성 파형 생성

In [ ]:
labels = hts.load(f"./downloads/jsut_ver1.1/basic5000/lab/{test_utt}.lab")

binary_dict, numeric_dict = hts.load_question_set(ttslearn.util.example_qst_file())

# 음소 연속 길이 예측
durations = predict_duration(
    device, labels, duration_model, duration_config, duration_in_scaler, duration_out_scaler,
    binary_dict, numeric_dict)

# 예측된 연속장을 풀 컨텍스트 레이블로 설정
labels.set_durations(durations)

# 음향 특징량 예측
out_feats = predict_acoustic(
    device, labels, acoustic_model, acoustic_config, acoustic_in_scaler,
    acoustic_out_scaler, binary_dict, numeric_dict)

# 음성 파형 생성
gen_wav = gen_waveform(
    sr, out_feats,
    acoustic_config.stream_sizes,
    acoustic_config.has_dynamic_features,
    acoustic_config.num_windows,
    post_filter=False,
)

In [ ]:
# 비교를 위해 원래 음성 로드
_sr, ref_wav = wavfile.read(f"./downloads/jsut_ver1.1/basic5000/wav/{test_utt}.wav")
ref_wav = (ref_wav / 32768.0).astype(np.float64)
ref_wav = librosa.resample(ref_wav, _sr, sr)

# 스펙트로그램 계산
spec_ref = librosa.stft(ref_wav, n_fft=fft_size, hop_length=hop_length, window="hann")
logspec_ref = np.log(np.abs(spec_ref))
spec_gen = librosa.stft(gen_wav, n_fft=fft_size, hop_length=hop_length, window="hann")
logspec_gen = np.log(np.abs(spec_gen))

mindb = min(logspec_ref.min(), logspec_gen.min())
maxdb = max(logspec_ref.max(), logspec_gen.max())

fig, ax = plt.subplots(2, 1, figsize=(8,6))
mesh = librosa.display.specshow(logspec_ref, hop_length=hop_length, sr=sr, cmap=cmap, x_axis="time", y_axis="hz", ax=ax[0])
mesh.set_clim(mindb, maxdb)
fig.colorbar(mesh, ax=ax[0], format="%+2.fdB")

mesh = librosa.display.specshow(logspec_gen, hop_length=hop_length, sr=sr, cmap=cmap, x_axis="time", y_axis="hz", ax=ax[1])
mesh.set_clim(mindb, maxdb)
fig.colorbar(mesh, ax=ax[1], format="%+2.fdB")

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Frequency [Hz]")
    
ax[0].set_title("Spectrogram of natural speech")
ax[1].set_title("Spectrogram of generated speech")

plt.tight_layout()

print("자연 음성")
IPython.display.display(Audio(ref_wav, rate=sr))
print("DNN 음성 합성")
IPython.display.display(Audio(gen_wav, rate=sr))

# 그림 6-13
savefig("./fig/dnntts_impl_tts_spec_comp")

### 평가 데이터에 대한 음성 파형 생성

#### 레시피의 stage 5 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 6 --stop-stage 6 --duration-model $duration_config_name --acoustic-model $acoustic_config_name \
    --tqdm $run_sh_tqdm 

## 자연 음성과 합성 음성의 비교 (bonus)

In [ ]:
from pathlib import Path
from ttslearn.util import load_utt_list

with open("./downloads/jsut_ver1.1/basic5000/transcript_utf8.txt") as f:
    transcripts = {}
    for l in f:
        utt_id, script = l.split(":")
        transcripts[utt_id] = script
        
eval_list = load_utt_list("data/eval.list")[::-1][:5]

for utt_id in eval_list:
    # ref file 
    ref_file = f"./downloads/jsut_ver1.1/basic5000/wav/{utt_id}.wav"
    _sr, ref_wav = wavfile.read(ref_file)
    ref_wav = (ref_wav / 32768.0).astype(np.float64)
    ref_wav = librosa.resample(ref_wav, _sr, sr)
    
    gen_file = f"exp/jsut_sr16000/synthesis_{duration_config_name}_{acoustic_config_name}/eval/{utt_id}.wav"
    _sr, gen_wav = wavfile.read(gen_file)
    print(f"{utt_id}: {transcripts[utt_id]}")
    print("자연 음성")
    IPython.display.display(Audio(ref_wav, rate=sr))
    print("DNN 음성 합성")
    IPython.display.display(Audio(gen_wav, rate=sr))

풀 컨텍스트 라벨이 아니고, 한자가 섞인 문장을 입력으로 한 TTS 의 구현은, `ttslearn.dnntts.tts` 모듈을 참조해 주세요. 이 장의 시작 부분에 제시된 학습된 모델을 사용하는 TTS는 해당 모듈을 사용합니다.

## 학습 완료 모델의 패키징 (bonus)

학습된 모델을 이용한 TTS에 필요한 파일을 모두 단일 디렉토리로 정리합니다.
'tslearn.dntts.DNTTS` 클래스에는, 정리한 디렉토리를 지정해, TTS를 실시하는 기능이 구현되고 있습니다.

### 레시피 stage 99 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 99 --stop-stage 99 --duration-model $duration_config_name --acoustic-model $acoustic_config_name

In [ ]:
!ls tts_models/jsut_sr16000_{duration_config_name}_{acoustic_config_name}

### 패키징된 모델을 이용한 TTS

In [ ]:
from ttslearn.dnntts import DNNTTS

# 패키징된 모델의 경로를 지정합니다.
model_dir = f"./tts_models/jsut_sr16000_{duration_config_name}_{acoustic_config_name}"
engine = DNNTTS(model_dir)
wav, sr = engine.tts("ここまでお読みいただき、ありがとうございました。")

fig, ax = plt.subplots(figsize=(8,2))
librosa.display.waveshow(wav.astype(np.float32), sr, ax=ax)
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Amplitude")
plt.tight_layout()

Audio(wav, rate=sr)

In [ ]:
if is_colab():
    from datetime import timedelta
    elapsed = (time.time() - start_time)
    print("소요시간:", str(timedelta(seconds=elapsed)))